<a href="https://colab.research.google.com/github/twyeh/highenergy/blob/main/Solutions_to_Problems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import flavio
import flavio.plots as fpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## $P_5'$: LHCb vs. SM

Find all LHCb measurements of $P_5'$ (or do it manually by looking at measurements.yml)

In [ ]:
meas = []
for mname, mobj in flavio.Measurement.instances.items():
    if mobj.experiment == 'LHCb':
        for obs in mobj.all_parameters:
            if isinstance(obs, tuple) and 'P5p' in obs[0]:
                meas.append(mname)
meas

In [ ]:
obs = '<P5p>(B0->K*mumu)';

_, bins = fpl.bin_plot_exp(obs, include_measurements=meas, col_dict={'LHCb': 'C0'});


In [ ]:
bins

Only some of those bins are reliable due to the presence of resonances

In [ ]:
bins_th = [
 (0.1, 0.98),
 (1.1, 2.5),
 (2.5, 4),
 (4, 6),
 (15, 19)]

In [ ]:
obs = '<P5p>(B0->K*mumu)';

_, bins = fpl.bin_plot_exp(obs, include_measurements=meas, col_dict={'LHCb': 'C0'});

fpl.bin_plot_th(obs, bins_th, threads=2)

plt.legend()
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

## $P_5'$: LHCb vs. NP

In [ ]:
wc = flavio.WilsonCoefficients()
wc.set_initial({'C9_bsmumu': -1.2}, 4.8)

In [ ]:
obs = '<P5p>(B0->K*mumu)';

_, bins = fpl.bin_plot_exp(obs, include_measurements=meas, col_dict={'LHCb': 'C0'});

fpl.bin_plot_th(obs, bins_th, threads=2, label='SM')
fpl.bin_plot_th(obs, bins_th, wc=wc, threads=2, fc='C1', alpha=0.2, label=r'$C_9=-1.2$')

plt.legend()
plt.xlabel(r'$q^2$ [GeV$^2$]')
plt.ylabel(flavio.Observable[obs].tex);

## Explaining flavour anomalies

In [ ]:
def prediction_sm_np(wc_dict, scale, obs, *args, **kwargs):
    wc = flavio.WilsonCoefficients()
    wc.set_initial(wc_dict, scale)
    sm = flavio.sm_prediction(obs, *args, **kwargs)
    np = flavio.np_prediction(obs, wc, *args, **kwargs)
    return sm, np

In [ ]:
prediction_sm_np({'C9_bsmumu': -1.2}, 4.8, '<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
prediction_sm_np({'C9_bsee': +1.2}, 4.8, '<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
prediction_sm_np({'C9_bsmumu': -1.2}, 4.8, '<Rmue>(B0->K*ll)', q2min=1.1, q2max=6)

In [ ]:
prediction_sm_np({'C9_bsee': +1.2}, 4.8, '<Rmue>(B0->K*ll)', q2min=1.1, q2max=6)

In [ ]:
prediction_sm_np({'CVL_bctaunutau': 0.21}, 4.8, 'Rtaul(B->Dlnu)')

In [ ]:
prediction_sm_np({'CVL_bctaunutau': 0.1}, 4.8, 'Rtaul(B->D*lnu)')

A SMEFT scenario giving a pattern that could solve all anomalies:

In [ ]:
import wcxf
wcxf.WC('SMEFT', 'Warsaw', 160, {'lq3_3323': -1/2500**2, 'lq3_2223': +1/25000**2}).match('WET', 'flavio')